# Find values in nested dictionary

20.03.20  
중첩된 딕셔너리에서 원하는 키값을 뽑아내야한다. 

In [1]:
import pandas as pd
import re
from ast import literal_eval
import warnings
warnings.filterwarnings(action='ignore')

파일 불러오기

In [2]:
log = pd.read_excel('log.xlsx')

### Keytalk

keytalk에 해당하는 값을 뽑아내야한다. (Keytalk안에 Keytalk가 있음)

데이터예시

In [3]:
list(log['keytalk_data'].sample())

['[{\\keytalk\\": [{\\"keytalk\\": \\"extraordinary\\"}], \\"vertical\\": \\"maimovie\\", \\"name\\": \\"extraordinary\\"}]"']

불필요한 문자 제거

In [4]:
#\
log['keytalk_new'] = log['keytalk_data'].str.replace('\\', '')


#[]
log['keytalk_new'] = log['keytalk_new'].apply(lambda x: x[1:-2])


# 맨 앞에 " 넣어주기
log['keytalk_new'] = log['keytalk_new'].str.replace('{k', '{"k')


# 딕셔너리로 만들기
log['keytalk_new'] = log['keytalk_new'].apply(lambda x: literal_eval(x) if x != '' else '')

키토크 뽑아내기

In [5]:
log['keytalk'] = ''

for i in range(0, len(log)):
    if log['keytalk_new'][i] != '':
        key_count = len(log['keytalk_new'][i])
        keytalk = []
        if type(log['keytalk_new'][i]) == tuple:
            for kc in range(0, key_count):
                temp = log['keytalk_new'][i][kc].get('keytalk')[0].get('keytalk')
                keytalk.append(temp)
        else:
            temp = log['keytalk_new'][i].get('keytalk')[0].get('keytalk')
            keytalk.append(temp)
        log['keytalk'][i] = keytalk

### Original

Sor_search안에 있는 search_morph 값

In [6]:
list(log['original_log'].sample())

['{\\bridge\\": \\"normal\\", \\"vertical_rank\\": [{\\"tcc_order\\": 1, \\"score\\": 22, \\"tcc_id\\": \\"maimovie\\", \\"rank\\": 1, \\"tcc_name\\": \\"Movie\\", \\"is_geodist\\": true}], \\"voice_text\\": \\"Historical documentary\\", \\"voice_analy\\": [], \\"voice_filter\\": [{\\"key\\": \\"maimovie\\", \\"value\\": [{\\"point_key\\": [\\"historical\\", \\"has_poster_img\\", \\"is_streamed_on_famous_service\\", \\"is_streamed\\", \\"is_won_all_awards\\"], \\"point_value\\": [50, 50, 50, 50, 50], \\"point_weight\\": [1, 1, 1, 1, 1], \\"point_check\\": [\\"historical\\"], \\"vertical\\": \\"maimovie\\", \\"name\\": \\"historical\\", \\"item_key\\": \\"filter_448\\"}], \\"key_name\\": \\"Movie\\"}], \\"voice_meta\\": [{\\"key\\": \\"maimovie\\", \\"value\\": [{\\"genre_ids\\": [\\"99\\"], \\"vertical\\": \\"maimovie\\", \\"name\\": \\"Documentary\\", \\"item_key\\": \\"meta_52216\\", \\"_vf_source\\": \\"admin\\"}], \\"key_name\\": \\"Movie\\"}], \\"voice_lang\\": \\"en\\", \\"voice_

In [7]:
def find_original(data):
    data = data.replace('\\', '')
    cc = re.compile('search_morph:\(.+\)')
    result = str(cc.findall(data))
    result = result.replace('[\'search_morph:(', '')
    result = result.replace(')\']', '')
    return result

In [8]:
log['original'] = log['original_log'].apply(lambda x: find_original(x))

### 결과

In [9]:
log[['keytalk', 'original']].sample(3)

,keytalk,original
149,,"""i"" OR ""watch"" OR ""a"" OR ""movie"" OR ""about"" O..."
271,,"""avatar"" OR ""the"" OR ""movie"""
385,[Science Fiction],"""find"" OR ""me"" OR ""sci"" OR ""fi"" OR ""movie"""
